# NIFTI Example - Whole Brain Segmentation
This notebook demonstrates how to process imaging data in MONAI Label and 3D Slicer. Before running this notebook, please first fun the Pre_NIFTI_egistration_Demo notebook to generate a sample data file. 

When launching the JupyterLab instance, make sure to select the **MONAI_ML** option in the Feature dropdown. 

For general examples of using Slicer within Jupyter notebook, please see: https://github.com/Slicer/SlicerNotebooks. For MONAI Label example notebooks and tutorials, see https://github.com/Project-MONAI/tutorials/tree/main/monailabel.


In [ ]:
import slicer, vtk
import JupyterNotebooksLib as slicernb
import os
from MONAILabel import MONAILabelLogic

### Download data from Project to the Execution Environment
To process your data saved on the platform in the notebook, the data must be available in the execution environment (as is the case with any DNAnexus app). You can download input data from a project to your notebook using dx download (bash) or download_dxfile (python) via in a notebook cell:

In [ ]:
!dx download subject01 -r

In [ ]:
DIR = os.path.join(os.getcwd(), "subject01")
os.makedirs(DIR, exist_ok=True)
os.environ["DIR"] = DIR

## 2. Prepare MONAI Label server

### Start the MONAI Label server (with MONAI Bundle app)

Download MONAI Bundle app and start the server. For the --studies flag, the command expects a path and will not execute correctly if we do not supply one. Even if you plan to upload volumes from Slicer, a directory is required. 

When the server is running, your terminal will now show the following message:

Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)

Please do not disturb the terminal from this state whilst you are still working with the MONAI Label server. 

Run following commands in a Terminal (separately, open a new Terminal tab in your JupyterLab session):
```bash
    monailabel apps --download --name monaibundle --output apps # Download monaibundle app
    monailabel start_server --app apps/monaibundle --studies subject01 --conf models wholeBrainSeg_Large_UNEST_segmentation  # Start the server 
```

## 2. Auto Segmentation with 3D Slicer (through SlicerJupyter)
Select **Slicer 5.8** kernel

### Connect to MONAI Label server

In [ ]:
SERVER_URL = "http://0.0.0.0:8000"


In [ ]:
ml = MONAILabelLogic()
ml.setServer(SERVER_URL)
ml.server_url

### Get server/app information

In [ ]:
ml.info()

In [ ]:
# Register local image with MONAI Label
image_file = "subject01/T1/orig_MNI.nii.gz"
image_id = "T1_MNI_orig"  # Give it a name

ml.upload_image(image_file, image_id=image_id)

### Query datastore (cases/images on the server)

In [ ]:
ml.datastore()

### Run Auto Segmentation

In [ ]:
# Run inference
label_file, params = ml.infer(
    model='wholeBrainSeg_Large_UNEST_segmentation',
    image_in=image_id
)


This will create a label file in a temporary directory. You can see this path location by yourself here:

In [ ]:
print(label_file)

## Visualise Image and Annotations

First you can print the volume sizes of your image and your label file to check the dimensions match. Then, view your newly created label.

In [ ]:
# Clear scene
slicer.mrmlScene.Clear(False)

# Load your T1 image
image_node = slicer.util.loadVolume('subject01/T1/orig_MNI.nii.gz')

# Load the NRRD label file
label_node = slicer.util.loadLabelVolume(label_file)

# Print sizes
print(f"T1 Volume size: {image_node.GetImageData().GetDimensions()}")
print(f"Label Volume size: {label_node.GetImageData().GetDimensions()}")

In [ ]:
slicernb.ViewDisplay("OneUpRedSlice")

## 3. Visualize and edit annotations with 3D Slicer

### Load image and label into 3D Slicer

### Visualize annotations

#### Static views

In [ ]:
# Convert label map to segmentation
segmentation_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(label_node, segmentation_node)

In [ ]:
# Show segmentation in 3D
segmentation_node.CreateClosedSurfaceRepresentation()

slicernb.ViewDisplay("FourUp")

#### (Optional) Update segment visibility (set to a specific segment)

In [ ]:
# Ensure segmentation node exists
segmentation_node = slicer.util.getNode('Segmentation') 

# Get segmentation object
segmentation = segmentation_node.GetSegmentation()

# Ensure display node exists
displayNode = segmentation_node.GetDisplayNode()
if not displayNode:
    slicer.modules.segmentations.logic().CreateDefaultDisplayNodes(segmentation_node)
    displayNode = segmentation_node.GetDisplayNode()

# Isolate Segment Visibility (Making ONLY the target segment visible)
target_segment_identifier = "14"  # Right-Cerebral-White-Matter, see apps/monaibundle/model/wholeBrainSeg_Large_UNEST_segmentation/docs/README.md
print(f"Isolating segment for visibility: {target_segment_identifier}")

for segmentID in segmentation.GetSegmentIDs():
    isVisible = (segmentation.GetSegment(segmentID).GetName() == target_segment_identifier)
    displayNode.SetSegmentVisibility(segmentID, isVisible)

##### Reset visibility to all segments

In [ ]:
segmentation = segmentation_node.GetSegmentation()
displayNode = segmentation_node.GetDisplayNode()

print(f"Resetting visibility to ALL segments")
for segmentID in segmentation.GetSegmentIDs():
    displayNode.SetSegmentVisibility(segmentID, True)

In [ ]:
# Lightbox view
slicernb.ViewLightboxDisplay("Red", rangeShrink=[45,50])

#### Dynamic views - Level 1. View objects + standard widgets
- Displayed content is saved in the notebook
- Views cannot be placed in a layout
- Low update rate (only for small adjustment of view parameters)
- Mouse and keyboard events are not captured

In [ ]:
# Slice view display
from ipywidgets import interact
@interact(position=(0,100))
def update(position=50):
    return slicernb.ViewSliceDisplay('Red', positionPercent=position)

#### Dynamic views - Level 2. View widgets
- Widgets can be placed in a layout
- Widget state (displayed content) is not saved in the notebook by default
- Low update rate (only for small adjustment of view parameters)
- Mouse and keyboard events are not captured

In [ ]:
# Slice widgets
from ipywidgets import HBox
slicernb.ViewSliceWidget('Red')
display(HBox([slicernb.ViewSliceWidget('Red'), slicernb.ViewSliceWidget('Yellow'), slicernb.ViewSliceWidget('Green')]))

In [ ]:
# 3D widget
display(HBox([slicernb.ViewSliceBaseWidget('Red', width="40%"), slicernb.View3DWidget(0, width="40%")]))

#### Dynamic views - Level 3. Interactive view widgets
- Some view controlling mouse and keyboard events are captured
- Only selected view can be displayed and controlled
- Medium update rate (somewhat usable on remote computers)

In [ ]:
# Adjust maximum rate of Slicer's Jupyter kernel consuming Jupyter messages.
# Lower values make the notebook more responsive but too low values may make the Slicer application
# slow to respond.
slicer.modules.jupyterkernel.setPollIntervalSec(0.001)

# Add image 3D display
# slicernb.showVolumeRendering(image_node)

# 3D view
slicernb.AppWindow.setWindowSize(scale=0.8)
live3d = slicernb.ViewInteractiveWidget('1')
live3d.trackMouseMove = False
display(live3d)

In [ ]:
# Slice view (use arrow keys to move between slices, right-click-and-drag to zoom in/out)
liveRedSlice = slicernb.ViewInteractiveWidget('R')
liveRedSlice.trackMouseMove = False
display(liveRedSlice)

### Edit Annotations

In [ ]:
# Setup Segment Editor
slicer.util.selectModule("SegmentEditor")
segmentEditorWidget = slicer.modules.segmenteditor.widgetRepresentation().self().editor
segmentEditorNode = segmentEditorWidget.mrmlSegmentEditorNode()
segmentEditorWidget.setSegmentationNode(segmentation_node)
segmentEditorWidget.setSourceVolumeNode(image_node)

segmentation = segmentation_node.GetSegmentation()
displayNode = segmentation_node.GetDisplayNode()

# Define the target segment
target_segment_identifier = "14" # Right-Cerebral-White-Matter, see apps/monaibundle/model/wholeBrainSeg_Large_UNEST_segmentation/docs/README.md

# Isolate Segment Visibility (Making ONLY the target segment visible)
print(f"Isolating segment for editing and visibility: {target_segment_identifier}")
for segmentID in segmentation.GetSegmentIDs():
    segment = segmentation.GetSegment(segmentID) 
    isVisible = (segment.GetName() == target_segment_identifier)
    displayNode.SetSegmentVisibility(segmentID, isVisible)

# Set the Active Segment for Editing
target_segment_ID = segmentation.GetSegmentIdBySegmentName(target_segment_identifier)
segmentEditorNode.SetSelectedSegmentID(target_segment_ID)
print(f"Set active editing segment to: {target_segment_identifier}")

# Configure viewer
app = slicernb.AppWindow(contents="viewers", windowScale=0.3)
slicernb.setViewLayout("OneUpRedSlice") # OneUpYellowSlice, OneUpGreenSlice, OneUp3D
app.setWindowSize(scale=0.5)
app.setContents("viewers")

# Create an interactive widget for the Red slice view ('R')
liveRedSliceSeg = slicernb.ViewInteractiveWidget('R') # other interactive widgets: Y, G, 1 (3D view)

liveRedSliceSeg.trackMouseMove = False
display(liveRedSliceSeg)

# Add buttons to switch between multiple effects
from ipywidgets import Button, HBox
cutButton = Button(description='Cut')
cutButton.on_click(lambda button: segmentEditorWidget.setActiveEffectByName("Scissors")) # 2D, 3D view
paintButton = Button(description='Paint')
paintButton.on_click(lambda button: segmentEditorWidget.setActiveEffectByName("Paint")) # 2D view
eraseButton = Button(description='Erase')
eraseButton.on_click(lambda button: segmentEditorWidget.setActiveEffectByName("Erase")) # 2D view
rotateButton = Button(description='Rotate')
rotateButton.on_click(lambda button: segmentEditorWidget.setActiveEffectByName("")) # 3D view
HBox([cutButton, paintButton, eraseButton, rotateButton])    

### Save/submit edited label

In [ ]:
labelmapVolumeNode = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLLabelMapVolumeNode')
slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(segmentation_node, labelmapVolumeNode, image_node)
slicer.util.saveNode(labelmapVolumeNode, label_file)

params = {"overwrite": True}
ml.save_label(image_id, label_file, params=params)


## 4. Active Learning
Train models anytime when new annotated labels are saved. MONAI Label server will fetch saved final ground truth label and fine-tune the prior model.

For each training loop, the new best metric model will be saved in **"\<BUNDLE_NAME\>/models/model.pt"**, in this use case, **"apps/monaibundle/model/wholeBrainSeg_Large_UNEST_segmentationmodel/models/model.pt"** is saved.
Note: runs only on GPU 

In [ ]:
# Run Train Task
ml.train_start(model="wholeBrainSeg_Large_UNEST_segmentation", params={'max_epochs': 20, 'run_id': 'training'})

In [ ]:
# Check Train Task Status
ml.train_status(check_if_running=False)

In [ ]:
# Stop any running Train Task(s)
ml.train_stop()

## 5. Repeat the Interactive Labeling Process
Repeat the above process of fetching data and active learning until all unlabeled data are annotated and trained.


## 6. Upload Data to the Project
For any data generated by your notebook that needs to be preserved, upload it to the project before the session ends and the JupyterLab worker terminates. 

In [ ]:
# Upload the labels
! dx upload -r subject01/labels

In [ ]:
# Upload updated model
! dx upload apps/monaibundle/model/wholeBrainSeg_Large_UNEST_segmentation/models/model.pt

Click Ctrl + C to interrupt the Monai Label Server in the terminal. 